In [1]:
import cv2
import numpy as np
 
 
def dodgeNaive(image, mask):
    # determine the shape of the input image
    width, height = image.shape[:2]
 
    # prepare output argument with same size as image
    blend = np.zeros((width, height), np.uint8)
 
    for col in range(width):
        for row in range(height):
            # do for every pixel
            if mask[col, row] == 255:
                # avoid division by zero
                blend[col, row] = 255
            else:
                # shift image pixel value by 8 bits
                # divide by the inverse of the mask
                tmp = (image[col, row] << 8) / (255 - mask)
                # print('tmp={}'.format(tmp.shape))
                # make sure resulting value stays within bounds
                if tmp.any() > 255:
                    tmp = 255
                    blend[col, row] = tmp
 
    return blend
 
 
def dodgeV2(image, mask):
    return cv2.divide(image, 255 - mask, scale=256)
 
 
def burnV2(image, mask):
    return 255 - cv2.divide(255 - image, 255 - mask, scale=256)
 
 
def rgb_to_sketch(src_image_name, dst_image_name, dataset_name):
    img_rgb = cv2.imread(src_image_name)
    cv2.imwrite(dataset_name, img_rgb)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    # 读取图片时直接转换操作
    # img_gray = cv2.imread('example.jpg', cv2.IMREAD_GRAYSCALE)
 
    img_gray_inv = 255 - img_gray
    img_blur = cv2.GaussianBlur(img_gray_inv, ksize=(21, 21),
                                sigmaX=0, sigmaY=0)
    img_blend = dodgeV2(img_gray, img_blur)
 
#     cv2.imshow('original', img_rgb)
#     cv2.imshow('gray', img_gray)
#     cv2.imshow('gray_inv', img_gray_inv)
#     cv2.imshow('gray_blur', img_blur)
#     cv2.imshow("pencil sketch", img_blend)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    cv2.imwrite(dst_image_name, img_blend)

In [2]:
# src_image_name = 'f:\\3.png'
# dst_image_name = 'f:\\sketch_example.jpg'
# rgb_to_sketch(src_image_name, dst_image_name)

In [3]:
import os
import glob
rootdir = r"d:\images\img" # 图片文件夹的路径
datadir = r"d:\images\dataset"
target = r"d:\images\new" # 转成线稿后的图片的路径，该文件需要提前创建
lists = os.listdir(rootdir)
imgType_list = {'jpg','bmp','png','jpeg','rgb','tif'}

j = 0

for i in range(len(lists)):
    path = os.path.join(rootdir, lists[i])
    
    tail = lists[i].split('.')[1]
    if(tail in imgType_list):
        target_name =  "d://images//new//" + str(i) + '.' + tail
        dataset_name = "d://images//dataset//" + str(i) + '.' + tail
        rgb_to_sketch(path, target_name, dataset_name)
        j += 1
        if j%20 == 0:
#             
#                 print(path)
            print(j/20)
        

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
d:\images\img\0923.jpeg
d:\images\img\0924.jpeg
d:\images\img\0925.png
d:\images\img\0926.jpeg
d:\images\img\0927.jpeg
d:\images\img\0928.jpeg
d:\images\img\0929.png
d:\images\img\0930.jpeg
d:\images\img\0931.jpeg
d:\images\img\0932.jpeg
d:\images\img\0933.jpeg
d:\images\img\0934.jpeg
d:\images\img\0935.jpeg
d:\images\img\0936.jpeg
d:\images\img\0937.jpeg
d:\images\img\0938.jpeg
d:\images\img\0939.jpeg
d:\images\img\0940.jpeg
d:\images\img\0941.jpeg
d:\images\img\0942.jpeg
47.0
d:\images\img\0943.jpeg
d:\images\img\0944.png
d:\images\img\0945.png
d:\images\img\0946.png
d:\images\img\0947.jpeg
d:\images\img\0948.jpeg
d:\images\img\0949.jpeg
d:\images\img\0950.jpeg
d:\images\img\0951.png
d:\images\img\0952.jpeg
d:\images\img\0953.png
d:\images\img\0954.jpeg
d:\images\img

d:\images\img\1265.png
d:\images\img\1266.png
d:\images\img\1267.jpg
d:\images\img\1268.png
d:\images\img\1269.jpg
d:\images\img\1270.jpg
d:\images\img\1271.jpg
d:\images\img\1272.png
d:\images\img\1273.jpg
d:\images\img\1274.png
d:\images\img\1275.png
d:\images\img\1276.png
d:\images\img\1277.jpg
d:\images\img\1278.png
d:\images\img\1279.jpg
d:\images\img\1280.jpg
d:\images\img\1281.png
d:\images\img\1282.jpg
64.0
d:\images\img\1283.jpg
d:\images\img\1284.png
d:\images\img\1285.png
d:\images\img\1286.jpg
d:\images\img\1287.png
d:\images\img\1288.png
d:\images\img\1289.jpg
d:\images\img\1290.jpg
d:\images\img\1291.png
d:\images\img\1292.jpg
d:\images\img\1293.jpg
d:\images\img\1294.png
d:\images\img\1295.png
d:\images\img\1296.png
d:\images\img\1297.jpg
d:\images\img\1298.jpg
d:\images\img\1299.jpg
d:\images\img\1300.jpg
d:\images\img\1301.jpg
d:\images\img\1302.jpg
65.0
d:\images\img\1303.jpg
d:\images\img\1304.jpg
d:\images\img\1305.jpg
d:\images\img\1306.jpg
d:\images\img\1307.jpg
d

d:\images\img\1621.jpg
d:\images\img\1622.jpg
81.0
d:\images\img\1623.jpg
d:\images\img\1624.jpg
d:\images\img\1625.jpg
d:\images\img\1626.jpg
d:\images\img\1627.jpg
d:\images\img\1628.jpg
d:\images\img\1629.jpg
d:\images\img\1630.jpg
d:\images\img\1631.jpg
d:\images\img\1632.jpg
d:\images\img\1633.jpg
d:\images\img\1634.jpg
d:\images\img\1635.jpg
d:\images\img\1636.jpg
d:\images\img\1637.jpg
d:\images\img\1638.jpg
d:\images\img\1639.jpg
d:\images\img\1640.jpg
d:\images\img\1641.jpg
d:\images\img\1642.jpg
82.0
d:\images\img\1643.jpg
d:\images\img\1644.jpg
d:\images\img\1645.jpg
d:\images\img\1646.jpg
d:\images\img\1647.jpg
d:\images\img\1648.jpg
d:\images\img\1649.jpg
d:\images\img\1650.jpg
d:\images\img\1651.jpg
d:\images\img\1652.jpg
d:\images\img\1653.jpg
d:\images\img\1654.jpg
d:\images\img\1655.jpg
d:\images\img\1656.jpg
d:\images\img\1657.jpg
d:\images\img\1658.jpg
d:\images\img\1659.jpg
d:\images\img\1660.jpg
d:\images\img\1661.jpg
d:\images\img\1662.jpg
83.0
d:\images\img\1663.